# 04. 영어/한국어 Word2Vec 훈련시키기
---
gensim 패키지에서 Word2Vec은 이미 구현되어져 있으므로 별도의 word2vec을 구현할 필요 없이 손쉽게 훈련 가능

In [4]:
# 1. 영어 Word2Vec 만들기
# 영어 데이터를 다운 받아 직접 word2vec 작업을 진행

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lemon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
# 훈련 데이터
# 링크 : https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip
# 위 파일 압축을 풀면 ted_en-2016408.xml 파일을 얻을 수 있음

datapath = 'D:/chch/startwhithtorch/datafolder/'
urllib.request.urlretrieve('https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml',
                           filename = datapath + 'ted_en-2016408.xml')

('D:/chch/startwhithtorch/datafolder/ted_en-2016408.xml',
 <http.client.HTTPMessage at 0x256b2ab1b80>)

In [7]:
# 해당 파일은 .xml로 우리가 원하는 자연어만 얻기 위해서는 전처리가 필요하다.
# 얻고자 하는 실질적인 데이터는 영어문장으로만 구성된 내용을 담고 있는
# <content> 와 </content> 사이의 내용이다.
# 추가로 사이의 내용에서 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 제거해야 한다.

### xml 파일 예시

<file id="1">  
  <head>  
<url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_how_to_avoid_them</url>  
       <pagesize>72832</pagesize>  
... xml 문법 중략 ...  
<content>  
Here are two reasons companies fail: they only do more of the same, or they only do what's new.  
To me the real, real solution to quality growth is figuring out the balance between two activities:  
... content 내용 중략 ...  
To me, the irony about the Facit story is hearing about the Facit engineers, who had bought cheap,   small electronic calculators in Japan that they used to double-check their calculators.   
(Laughter)  
... content 내용 중략 ...  
(Applause)  
</content>  
</file>  
<file id="2">  
    <head>  
<url>http://www.ted.com/talks/lisa_nip_how_humans_could_evolve_to_survive_in_space<url>  
... 이하 중략 ...  

In [8]:
# 훈련 데이터 전처리하기
targetXML = open(datapath + 'ted_en-2016408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

parse_text = '/n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'/([^)]*/)', '', parse_text)
# parse_text 를 불러와서 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거한다는 뜻

In [9]:
len(content_text)

23430418

In [10]:
sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용해 문장 토큰화(sentence tokenize) 진행

In [11]:
sent_text[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new.",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation.',
 'Both are necessary, but it can be too much of a good thing.',
 'Consider Facit.',
 "I'm actually old enough to remember them."]

In [12]:
normalized_text = []
for string in sent_text :
    tokens = re.sub(r'[^a-z0-9]+', ' ', string.lower())
    normalized_text.append(tokens)
# 각 문장에서 알파벳소문자와 숫자를 제외하고는 블랭크로 바꿈

In [13]:
normalized_text[:5]

['here are two reasons companies fail they only do more of the same or they only do what s new ',
 'to me the real real solution to quality growth is figuring out the balance between two activities exploration and exploitation ',
 'both are necessary but it can be too much of a good thing ',
 'consider facit ',
 'i m actually old enough to remember them ']

In [14]:
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용해 단어 토큰화 수행

In [15]:
result[:5]

[['here',
  'are',
  'two',
  'reasons',
  'companies',
  'fail',
  'they',
  'only',
  'do',
  'more',
  'of',
  'the',
  'same',
  'or',
  'they',
  'only',
  'do',
  'what',
  's',
  'new'],
 ['to',
  'me',
  'the',
  'real',
  'real',
  'solution',
  'to',
  'quality',
  'growth',
  'is',
  'figuring',
  'out',
  'the',
  'balance',
  'between',
  'two',
  'activities',
  'exploration',
  'and',
  'exploitation'],
 ['both',
  'are',
  'necessary',
  'but',
  'it',
  'can',
  'be',
  'too',
  'much',
  'of',
  'a',
  'good',
  'thing'],
 ['consider', 'facit'],
 ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them']]

In [16]:
print(f'총 샘플의 개수: {len(result)}')

총 샘플의 개수: 265335


In [18]:
# 3) Word2Vec 훈련시키기
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(
                 sentences = result,    # 훈련시킬 문장셋
                                        # 각 문장마다 하나의 토큰 list를 생성하며 토큰 list의 개수는 문장 개수 n개 만큼 생성하여 sentences에 저장함
                 vector_size = 100,     # 워드 벡터의 특징 값, 워드 벡터의 차원
                 window = 5,            # 현재 단어와 예측 단어의 최대 거리
                 min_count = 5,         # 이 수보다 낮은 빈도수의 단어는 무시
                 workers = 4,           # 모델 생성시 사용할 쓰레드 개수
                 sg = 0                 # 1 = skip-gram / 0 = CBOW
                 )

In [19]:
# 학습을 했으니 이용해보자

# .wv.most_similar()로 가장 유사한 단어들을 출력해보자
model_result = model.wv.most_similar('woman')
print(model_result)

[('man', 0.8601292967796326), ('girl', 0.8386016488075256), ('lady', 0.8307408094406128), ('boy', 0.7759738564491272), ('soldier', 0.7337160110473633), ('child', 0.7305009365081787), ('kid', 0.7267239093780518), ('gentleman', 0.7170665264129639), ('doctor', 0.7078558802604675), ('guy', 0.7008314728736877)]


In [20]:
# 4) Word2Vec 모델 저장하고 로드하기
# 학습한 모델을 언제든 다시 사용할 수 있도록 저장해보자

# 모델 저장
model.wv.save_word2vec_format('./eng_w2v')
# 모델 로드
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v')

In [21]:
# 저장한 모델 불러와서 유사한 단어 뽑아보기
model_result = loaded_model.most_similar('sky')
print(model_result)

[('sun', 0.7738822102546692), ('river', 0.7321430444717407), ('sea', 0.7295847535133362), ('ocean', 0.7232835292816162), ('mountain', 0.7224076390266418), ('sand', 0.721751868724823), ('moon', 0.7213958501815796), ('desert', 0.7180595993995667), ('window', 0.7092677354812622), ('lake', 0.7088762521743774)]


## 2. 한국어 Word2Vec 만들기
---

In [ ]:
# 위키피티아 한국어 덤프 파일을 다운 받아서 한국어로 w2v을 진행해보자

In [22]:
# 1) 위키피디아 한국어 덤프 파일 다운로드
# 링크: https://dumps.wikimedia.org/kowiki/latest/
# 사용할 파일: kowiki-latest-pages-articles.xml.bz2

# >>> wikiko 다운 받았음

In [23]:
# 2) 위키피디아 익스트랙터 다운로드

# >>> wikiko 다운 받았음

In [1]:
# 3) 위키피디아 한국어 덤프 파일 변환
# 위 두개의 파일들을 같은 폴더에 위치시킨 후에 아래 명령어를 실행
# python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2 

# ----------------------------------------------------------
# Wikiextractor.py 파일 변경
# from multiprocessing import Queue, Process, cpu_count
# ▼
# from multiprocessing import Queue, Process, cpu_count
# from multiprocessing.dummy import Queue, Process

# 186
# return open(filename, 'w')
# >>
# return open(filename, 'w', encoding='utf-8')

# 210
# output = open(output_file, 'w')
# >>>
# output = open(output_file, 'w', encoding='utf-8')

# 드디어 해결!!!!!!!! 우예히끼항!

In [2]:
# 4) 훈련 데이터 만들기
# AA디렉토리 안의 모든 파일인 wiki00~wiki90에 대해서 wikiAA.txt로 통합해보자

# cmd 에서 아래 커맨드 실행
# D:/chch/wikiko/text>  # 경로로 들어가서
# copy AI wikiAI.txt    # 이렇게 AA~AI까지

# 그 다음 이 6개 파일도 하나로 합치자
# D:/chch/wikiko/text>        # 경로에서
# copy wikiA* wiki_data.txt   # 하나로 합쳐

# 훈련 데이터 완성!!!

In [1]:
# 5) 훈련 데이터 전처리 하기

f = open('D:/chch/wikiko/text/wiki_data.txt', encoding='utf-8')

In [2]:
# 파일이 제대로 불러와졌는지 확인
i = 0
while True:
    line = f.readline()
    if line != '/n':
        i += 1
        print('%d번째 줄: '%i + line)
    if i == 5 :
        break

f.close()

1번째 줄: <doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄: 지미 카터

3번째 줄: 제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.

4번째 줄: 생애.

5번째 줄: 어린 시절.



In [3]:
from konlpy.tag import Okt  
okt=Okt()

fread = open('D:/chch/wikiko/text/wiki_data.txt', encoding='utf-8')
# 파일을 다시 처음부터 읽음.
n=0
result = []

while True:
    line = fread.readline() #한 줄씩 읽음.
    if not line: break # 모두 읽으면 while문 종료.
    n=n+1
    if n%5000==0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d번째 While문."%n)
    tokenlist = okt.pos(line, stem=True, norm=True) # 단어 토큰화
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함

    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
      result.append(temp) # 결과에 저장
      
fread.close()


5000번째 While문.
10000번째 While문.
15000번째 While문.
20000번째 While문.
25000번째 While문.
30000번째 While문.
35000번째 While문.
40000번째 While문.
45000번째 While문.
50000번째 While문.
55000번째 While문.
60000번째 While문.
65000번째 While문.
70000번째 While문.
75000번째 While문.
80000번째 While문.
85000번째 While문.
90000번째 While문.
95000번째 While문.
100000번째 While문.
105000번째 While문.
110000번째 While문.
115000번째 While문.
120000번째 While문.
125000번째 While문.
130000번째 While문.
135000번째 While문.
140000번째 While문.
145000번째 While문.
150000번째 While문.
155000번째 While문.
160000번째 While문.
165000번째 While문.
170000번째 While문.
175000번째 While문.
180000번째 While문.
185000번째 While문.
190000번째 While문.
195000번째 While문.
200000번째 While문.
205000번째 While문.
210000번째 While문.
215000번째 While문.
220000번째 While문.
225000번째 While문.
230000번째 While문.
235000번째 While문.
240000번째 While문.
245000번째 While문.
250000번째 While문.
255000번째 While문.
260000번째 While문.
265000번째 While문.
270000번째 While문.
275000번째 While문.
280000번째 While문.
285000번째 While문.
290000번째 While문.
295000번째 While문.
300000번째 While문.


KeyboardInterrupt: 

In [4]:
# 너무 오래 걸려서 멈춤!
fread.close()

In [7]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 1860178


In [10]:
# 6) Word2Vec 훈련시키기
from gensim.models import Word2Vec
model = Word2Vec(result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [11]:
model_result1 = model.wv.most_similar('대한민국')
print(model_result1)

[('한국', 0.6795631647109985), ('우리나라', 0.5766524076461792), ('조선민주주의인민공화국', 0.5732355713844299), ('재외동포', 0.5540997982025146), ('부산광역시', 0.5316900610923767), ('국내', 0.5233590602874756), ('관세청', 0.516149640083313), ('대구광역시', 0.4936351776123047), ('방위사업청', 0.49209317564964294), ('여성가족부', 0.49202823638916016)]


In [12]:
model_result2 = model.wv.most_similar('고양이')
print(model_result2)

[('토끼', 0.810920238494873), ('강아지', 0.7959395051002502), ('애완동물', 0.7670261263847351), ('거북이', 0.7571203708648682), ('거위', 0.7559707164764404), ('애완견', 0.741983950138092), ('울음소리', 0.7234353423118591), ('사냥개', 0.716405987739563), ('개구리', 0.7151011228561401), ('햄스터', 0.6961157917976379)]


## 3. 사전 훈련된 Word2Vec 임베딩(Pre-trained Word2Vec embedding) 소개
---

In [13]:
# 방대한 데이터로 사전에 훈련된 워드 임베딩을 가지고 와서 해당 벡터들의 값을 원하는 작업에 사용 가능

In [15]:
# 1) 영어

# 구글이 제공하는 사전 훈련된 W2V 모델을 사용해보자
google_w2v = 'D:/chchdata/dataset/GoogleNews-vectors-negative300.bin'

In [16]:
import gensim

# 모델을 로드 
model = gensim.models.KeyedVectors.load_word2vec_format(google_w2v, binary=True)

In [17]:
# 모델 크기 확인
print(model.vectors.shape)

(3000000, 300)


In [18]:
# 모델 크기는 3,000,000 * 300 인데 3백만 개의 단어와 각 단어의 차원은 300이라는 뜻

In [21]:
# 두 단어의 유사도 계산하기
print(model.similarity('this', 'is'))
print(model.similarity('book', 'books'))
print(model.similarity('book', 'duck'))

0.40797037
0.73791784
0.041631036


In [22]:
# 단어 book의 벡터 출력
print(model['book'])

[ 0.11279297 -0.02612305 -0.04492188  0.06982422  0.140625    0.03039551
 -0.04370117  0.24511719  0.08740234 -0.05053711  0.23144531 -0.07470703
  0.21875     0.03466797 -0.14550781  0.05761719  0.00671387 -0.00701904
  0.13183594 -0.25390625  0.14355469 -0.140625   -0.03564453 -0.21289062
 -0.24804688  0.04980469 -0.09082031  0.14453125  0.05712891 -0.10400391
 -0.19628906 -0.20507812 -0.27539062  0.03063965  0.20117188  0.17382812
  0.09130859 -0.10107422  0.22851562 -0.04077148  0.02709961 -0.00106049
  0.02709961  0.34179688 -0.13183594 -0.078125    0.02197266 -0.18847656
 -0.17480469 -0.05566406 -0.20898438  0.04858398 -0.07617188 -0.15625
 -0.05419922  0.01672363 -0.02722168 -0.11132812 -0.03588867 -0.18359375
  0.28710938  0.01757812  0.02185059 -0.05664062 -0.01251221  0.01708984
 -0.21777344 -0.06787109  0.04711914 -0.00668335  0.08544922 -0.02209473
  0.31835938  0.01794434 -0.02246094 -0.03051758 -0.09570312  0.24414062
  0.20507812  0.05419922  0.29101562  0.03637695  0.04

In [1]:
# 2) 한국어
import gensim

ko_pre_path = 'D:/chchdata/dataset/kowiki_pre_w2v.bin'

In [2]:
# 불러올 때 에러 발생
# >> AttributeError: Can't get attribute 'Vocab' on <module 'gensim.models.word2vec' from 'C:\\Users\\lemon\\anaconda3\\envs\\chch\\lib\\site-packages\\gensim\\models\\word2vec.py'>

# gensim 버전이 4점대라서 그런거니 3점대로 내리자
# 원래 4.1.2
# pip install --upgrade --user gensim==3.8.3

# 해결!

In [3]:
# 저장한 파일을 모델로 지정
model = gensim.models.Word2Vec.load(ko_pre_path)

In [5]:
result=model.wv.most_similar("강아지")
print(result)

[('고양이', 0.7290452718734741), ('거위', 0.7185635566711426), ('토끼', 0.7056223154067993), ('멧돼지', 0.6950401067733765), ('엄마', 0.6934334635734558), ('난쟁이', 0.6806551218032837), ('한마리', 0.6770296096801758), ('아가씨', 0.6750352382659912), ('아빠', 0.6729634404182434), ('목걸이', 0.6512460708618164)]
